Ever what happens when customers don't make good on their bank loans? Once a bank has decided that a debt is uncollectabled, the bank doens't just forfeit the money they are owed. Instead the bank will assign a score to the account which will assess how much money the bank is exected to recover from the customer eventually. The assessment score is a function of the total debt, the probability of the customer paying and many other features that can impact the customers ability to repay the debt. 

The more a bank expects to recover the mroe effort the bank is willing to expend in order to recoer it through continuious contact with the debtor. IF the debt to be recovered is not very large then a bank can just add the customer to an automatic dialing or emailing system. Howeever for larger values, broken up into thresholds, the cost put into recovery will increase by 50$

This analysis will investigate if the debt recovered at a higher threshold exceedd the added costs in recoering it. 

In [ ]:
# imports
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

# import stats module
from scipy import stats

# read in csv
df = pd.read_csv('datasets/bank_data.csv')

# print the first 5 rows of the df
df.head()

Although banks set several thresholds and different strategies for each this analysis will focus on the first threshold, Level 0 and Level 1 as defined below:
* Level 0: 0 < Expected Recovery <= 1000
* Level 1: 1001 < Expected Recover <= 2000

We will work to identify if there are any other factors besides the amount the bank expects to recover to distinguish between a level 0 effort or a level 1 effort. For example, is there a difference in age betweent he debtors on level 0 compared to level 1? To investigate we will construct a scatter plot of age as a function of Expected REcovery for the range of 0 - 2000, which covers level 0 and level 1 

In [ ]:
# scatter plot of age vs expected recovery
plt.scatter(x=df['expected_recovery_amount'], y=df['age'], c="g", s=2)
plt.xlim(0, 2000)
plt.ylim(0, 60)
plt.xlabel('Expected Recovery Amount')
plt.ylabel('Age')
plt.legend(loc=2)
plt.axvline(x=1000, ymin=0, ymax=60, color = 'black')
plt.show()

In order to confidently state that a bank's recovery strategy is due to the expected recovery amount we need to rule out other factors. In a sense we want to rule out a statistical difference in factors such as age, or sex of debtors in level 0 and level 1. 

When examining the plot above there is not an glaring difference in age between level 0 and level 1. In order to determine that there is no statically significant difference in age we will conduct the Kruskal-Wallis test, of the distribution of ages for level and level 1 debtors 

In [ ]:
# compute average age just below and above the threshold
era_900_1100 = df.loc[(df['expected_recovery_amount']<1100) & 
                      (df['expected_recovery_amount']>=900)]
by_recovery_strategy = era_900_1100.groupby(['recovery_strategy'])
by_recovery_strategy['age'].describe().unstack()

# perform Kruskal-Wallis test 
Level_0_age = era_900_1100.loc[df['recovery_strategy']=="Level 0 Recovery"]['age']
Level_1_age = era_900_1100.loc[df['recovery_strategy']=="Level 1 Recovery"]['age']
stats.kruskal(Level_0_age,Level_1_age) 

From the Kruskal Result we receive a pvalue of 0.0629 which when compared with a alpha value of 0.05 we would fail to reject that the age distribution of the levels are different. 

Now to approach the same issue but for sex of the debtors in each level. we can address this questions statistically by construction a chi-square test of the percentage of cutomers that are female vs male.

In [ ]:
# no. of customers in each category
crosstab = pd.crosstab(df.loc[(df['expected_recovery_amount']<1100) & 
                              (df['expected_recovery_amount']>=900)]['recovery_strategy'], 
                       df['sex'])
print(crosstab)

# chi-square test
chi2_stat, p_val, dof, ex = stats.chi2_contingency(crosstab)
print(p_val)

Now that we have are reaconably assured that customers in level 0 and level 1 have similar age and gender breakdowns we can focus on the actual recovery amount. 

Next week I willl begin exaining the relationship between actual recovered amount and expected recovery amount.